In [5]:
# Package imports
import os
import pickle
import xml.etree.ElementTree as ET
import random

In [6]:
class DataProcess():
    def __init__(self):
        self.rootDir = "./data"
        self.batch_pointer = 0
        self.batch_size = 2
        
        stroke_dir = self.rootDir
        data_file = os.path.join(self.rootDir, "strokes_training_data.cpkl")
        self.process(stroke_dir, data_file)
    
    # Read processed data from .cpkl file.
    def read_processed(self, data_file):
        # Opening in read mode
        f = open(data_file, 'rb')
        stroke_data = pickle.load(f)
        f.close()
        print(stroke_data[0])
        
        # TODO : Do normalisation here.
        self.num_batches = int(len(stroke_data)/self.batch_size)
        print("Loaded dataset with {} data examples").format(len(stroke_data))
        print("Batch size for dataset : {}").format(self.num_batches)
        

        
    def process(self, rootDir, data_file):
        # Function that outputs linestrokes given filepath.    
        def convert_linestroke_file_to_array(filepath):
            strokeFile = ET.parse(filepath)
            root = strokeFile.getroot()
            x_min_offset = -1000000
            y_min_offset = -1000000
            y_height = 0
            for i in range(1,4):
                x_min_offset = min(x_min_offset, float(root[0][i].attrib['x']))
                y_min_offset = min(y_min_offset, float(root[0][i].attrib['y']))
                y_height = max(y_height, float(root[0][i].attrib['y']))
            #TODO(add normalization)
            y_height -= y_min_offset
            x_min_offset -=100
            y_min_offset -=100
            strokeSet = root[1]
            allStrokes = []
            for i in range(len(strokeSet)):
                points = []
                for point in strokeSet[i]:
                    points.append([(float(point.attrib['x']) - x_min_offset), (float(point.attrib['y']) - y_min_offset)])
                allStrokes.append(points)
            return allStrokes    
                
    
        def get_all_files():
            rootDir = "./data"
            filePaths = []
            for dirpath, dirnames, filenames in os.walk(rootDir):
                for file in filenames:
                    print (file)
                    filePaths.append(os.path.join(dirpath, file))
            return filePaths

    #for file in get_all_files(rootdir):
     #   convert_linestroke_file_to_array(file)
        
    # Function to convert strokes to inputStrokeMatrix
        def strokes_to_input_matrix(strokes):
            strokeMatrix = []
            prev_x = 0
            prev_y = 0
            for stroke in strokes:
                for num_point in range(len(stroke)):
                    x = stroke[num_point][0] - prev_x
                    y = stroke[num_point][1] - prev_y
                    prev_x = stroke[num_point][0]
                    prev_y = stroke[num_point][1]
                    z = 0
                    if (num_point == len(stroke)-1):
                        z = 1
                    example = [x,y,z]
                    strokeMatrix.append(example)
            return strokeMatrix
        
        allFiles = get_all_files()
        strokes = []
        counter = 0
        for file in allFiles:
            if file[-3:] == "xml":
                counter = counter + 1
                stroke = strokes_to_input_matrix(convert_linestroke_file_to_array(file))
                strokes.append(stroke)
            assert len(strokes) == counter    
        f = open(data_file,"wb")
        pickle.dump([strokes], f, protocol=2)
        f.close()
        print("Saved {} lines", len(strokes))

def next_batch():
    
                

In [7]:
d = DataProcess()


strokes_training_data.cpkl
.DS_Store
.DS_Store
a03-020.txt
a03-030.txt
a03-027.txt
a03-009.txt
a03-011.txt
a03-014.txt
a03-054.txt
a03-193z.txt
a03-040.txt
a03-068.txt
a03-063.txt
a03-200z.txt
a03-059.txt
a03-206z.txt
a03-023.txt
a03-066.txt
a03-218z.txt
a03-037.txt
a03-017.txt
j06-316z.txt
j06-278z.txt
j06-328z.txt
j06-312z.txt
j06-308z.txt
j06-294z.txt
j06-333z.txt
j06-287z.txt
j06-323z.txt
j06-301z.txt
d10-681z.txt
d10-703z.txt
d10-718z.txt
d10-697z.txt
d10-709z.txt
d10-684z.txt
k10-058z.txt
k10-067z.txt
k10-086z.txt
k10-075z.txt
k10-114z.txt
k10-095z.txt
k10-046z.txt
k10-051z.txt
k10-103z.txt
a07-457z.txt
a07-421z.txt
a07-431z.txt
a07-472z.txt
a07-480z.txt
a07-466z.txt
a07-438z.txt
a07-446z.txt
a07-413z.txt
p04-422z.txt
p04-401z.txt
p04-478z.txt
p04-431z.txt
p04-460z.txt
p04-455z.txt
p04-445z.txt
p04-373z.txt
p04-394z.txt
p04-504z.txt
p04-483z.txt
p04-379z.txt
p04-387z.txt
p04-468z.txt
p04-411z.txt
p04-416z.txt
p04-438z.txt
p04-492z.txt
n07-889z.txt
n07-860z.txt
n07-754z.txt
n07-85

In [8]:
d.read_processed("data/strokes_training_data.cpkl")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



AttributeError: 'NoneType' object has no attribute 'format'